In [ ]:
%load_ext autoreload
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (9, 6)
import pywt
import random
import numpy as np
import sys
sys.path.append('../ecg')

## Loading Data

In [ ]:
LOAD_FROM_FILE = False
LOAD_FROM_MODEL = True
model_path = "../saved/3.1/1485643942/0.317-0.908-033-0.222-0.927.hdf5"

import load
import json
import util

args = util.get_object_from_dict(data_path="../data")

if LOAD_FROM_FILE is True:
    params = json.load(open('../configs/vanilla_load.json', 'r'))
elif LOAD_FROM_MODEL is True:
    params = util.get_model_params(model_path)
else:
    params = {"step": 128, "toy": True}
    
dl = load.load(args, params)
print(dl.class_to_int)

In [ ]:
predictions = np.load(open(util.get_prediction_path_for_model(model_path, 'test'), 'rb'))

In [ ]:
x = dl.x_test
y = dl.y_test

def from_one_hot_to_int(label):
    return np.argmax(label, axis=-1)

def get_x_y_predictions_at_index(index):
    x_sample = x[index]
    y_sample = from_one_hot_to_int(y[index])
    y_prediction = np.argmax(predictions[index], axis=-1)
    return x_sample, y_sample, y_prediction

def get_sample_from_class(category_name, min_mistakes = 20, num_tries = 1000):
    class_needed = dl.class_to_int[category_name]
    indices = np.where(np.sum(np.argmax(y, axis=-1) == class_needed, axis=1) > 0)[0]
    for _ in range(num_tries):
        index = random.choice(indices)
        x_sample, y_sample, y_prediction = get_x_y_predictions_at_index(index)
        num_wrong = np.sum(y_sample != y_prediction)
        if (num_wrong > min_mistakes):
            break
    return x_sample, y_sample, y_prediction, num_wrong

x_sample, y_sample, y_prediction, num_wrong = get_sample_from_class(u'JUNCTIONAL')
print("Prediction got wrong " +  str(num_wrong * 1.0 / len(y_sample)))

In [ ]:
import matplotlib.cm as cm

def from_int_to_name(l):
    return dl.classes[l]

def draw_sample(x_sample, y_sample, y_prediction):
    colors = cm.rainbow(np.linspace(0, 1, 20))
    y_times = np.linspace(0, len(x_sample), len(y_sample))
    for y_chosen in [y_sample]:
        for y_uniq in np.unique(y_chosen):
            plt.vlines(
                y_times[y_chosen == y_uniq],
                min(x_sample),
                max(x_sample),
                label=from_int_to_name(y_uniq),
                color=colors[y_uniq],
                alpha=1
            )
        y_times += 20
    print(np.array(dl.classes)[y_sample])
    print(np.array(dl.classes)[y_prediction])
    plt.plot(x_sample, color='#999999', alpha=1)
    plt.legend()
    plt.show()


draw_sample(x_sample, y_sample, y_prediction)